In [3]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import time
import requests
from requests import get
#import regex as re
from requests import get
from bs4 import BeautifulSoup
#from wordcloud import WordCloud, STOPWORDS
import timeit
import nltk
from nltk.util import *
from nltk.corpus import stopwords
import re
import seaborn as sns
from collections import defaultdict

In [4]:

url = 'https://photo.stackexchange.com/questions'
time.sleep(5)
# Getting the response from the source URL.
response = get(url)
print(response.text[:500])


<!DOCTYPE html>


    <html class="html__responsive">

    <head>

        <title>Newest Questions - Photography Stack Exchange</title>
        <link rel="shortcut icon" href="https://cdn.sstatic.net/Sites/photo/Img/favicon.ico?v=20ef505cc470">
        <link rel="apple-touch-icon" href="https://cdn.sstatic.net/Sites/photo/Img/apple-touch-icon.png?v=0be01e9424b4">
        <link rel="image_src" href="https://cdn.sstatic.net/Sites/photo/Img/apple-touch-icon.png?v=0be01e9424b4"> 
        


In [5]:

# Parsing respose.txt by creating a BeautifulSoup object and assigning it to htlm_soup.
time.sleep(5)
html_soup = BeautifulSoup(response.text, 'html.parser')

In [6]:
Questions_EachPage = html_soup.find_all('div', class_ = 'question-summary')
print(type(Questions_EachPage))
print(len(Questions_EachPage))

<class 'bs4.element.ResultSet'>
50


In [7]:
from urllib.request import urlopen 
from socket import timeout

url = "https://photo.stackexchange.com/questions?tab=newest&pagesize=50"
try: 
    string = urlopen(url, timeout=5).read()
except ConnectionResetError:
    print("==> ConnectionResetError")
    pass
except timeout: 
    print("==> Timeout")
    pass

In [8]:
pages_list=[]                         # Empty list that will be appended with the page count

pages_list.append("https://photo.stackexchange.com/questions?tab=newest&pagesize=50")

for page in list(range(2, 601)):     # Looping over first 400 webpages

  pages_list.append('https://photo.stackexchange.com/questions?tab=newest&page='+str(page))
  
print(len(pages_list))

600


In [9]:
questions_list=[]                     # Creating an empty list of questions that will be appended with the question counts.

# time.sleep(5)
for x in pages_list:                  # Looping over all the pages in the page list
  page = requests.get(x)
  soup = BeautifulSoup(page.text, 'html.parser')

  question_name = soup.find_all('h3')   # Finding the h3 tag that has details of the question
  for question in question_name:
        if question.find('a'):
            questions_list.append(question.find('a').text)

In [10]:
print(len(questions_list))
print(questions_list[:11])

25465
['current community', 'more stack exchange communities', 'What should I be careful about while buying used mirrorless camera and lenses?', 'How can I make part of an image transparent in Lightroom?', 'Confused about Lightroom Smart Collection conditions', "Is there a technical reason for modern on-camera flashes' flash duration?", 'Fast Prime Wide lens for Landscape Photography and focal distance [duplicate]', 'How to shortcut the “release/focus priority” on the FujiFilm X-T4?', 'Blur effect or out of focus?', 'Using Nikon COOLPIX L840 as a webcam [closed]', 'How to improve my jewellery photography?']


In [11]:
# To get the tags from the webpage

def ListToString(string):
  # initialize an empty string 
  string1 = ""
    
  # traverse in the string   
  for element in string:  
      string1 = string1 + element   
     
  return string1


tag=[]

for x in pages_list:
  page = requests.get(x)
  soup = BeautifulSoup(page.text, 'html.parser')

  tags=[]

  for div in soup.find_all('div', {"class": re.compile("^tags")}):      
    tags.append(div.get('class')[1:])



  clean_tags=[]

  for i in tags:
    clean_tags.append(ListToString(i).replace('t-','|'))



  for j in clean_tags:
    tag.append(j[1:])

In [12]:
print(len(tag))
print(tag[:11])

24265
['equipmen|recommendation|fujifilm|used-equipment', 'lightroom|photo-editing', 'lightroom|keywords|smar|collections', 'flash|hotshoe-flash|vintage|sync-speed', 'landscape|prime|focal-distance', 'camera-settings|fujifilm', 'effect|blur', 'nikon|live-view', 'lighting|produc|photography', 'lens|error|canon-5d-mark-iii', 'nikon|image-quality|nikon-d5600']


In [13]:

# Creating a dataframe for questions and tags

ask_photo_df = pd.DataFrame(list(zip(tag, questions_list)), columns =['Tags', 'Questions']) 
ask_photo_df.head(10)

,Tags,Questions
0,equipmen|recommendation|fujifilm|used-equipment,current community
1,lightroom|photo-editing,more stack exchange communities
2,lightroom|keywords|smar|collections,What should I be careful about while buying us...
3,flash|hotshoe-flash|vintage|sync-speed,How can I make part of an image transparent in...
4,landscape|prime|focal-distance,Confused about Lightroom Smart Collection cond...
5,camera-settings|fujifilm,Is there a technical reason for modern on-came...
6,effect|blur,Fast Prime Wide lens for Landscape Photography...
7,nikon|live-view,How to shortcut the “release/focus priority” o...
8,lighting|produc|photography,Blur effect or out of focus?
9,lens|error|canon-5d-mark-iii,Using Nikon COOLPIX L840 as a webcam [closed]


In [14]:

ask_photo_df['Questions'] = ask_photo_df['Questions'].str.replace('()','')
ask_photo_df['Questions'] = ask_photo_df['Questions'].str.replace(';','')
ask_photo_df['Questions'] = ask_photo_df['Questions'].str.replace(':','')
ask_photo_df['Questions'] = ask_photo_df['Questions'].str.replace('/','')


ask_photo_df['Tags'] = ask_photo_df['Tags'].str.replace('()','')
ask_photo_df['Tags'] = ask_photo_df['Tags'].str.replace(';','')
ask_photo_df['Tags'] = ask_photo_df['Tags'].str.replace(':','')
ask_photo_df['Tags'] = ask_photo_df['Tags'].str.replace('û','.')
ask_photo_df['Tags'] = ask_photo_df['Tags'].str.replace('\t','')

In [15]:
ask_photo_tags = ' '.join(ask_photo_df['Tags'].str.lower())

In [17]:
ask_photo_df["Tags"] = [item.replace("|", " __label__") for item in ask_photo_df["Tags"]]
ask_photo_df['Tags'] ='__label__' + ask_photo_df['Tags'].astype(str)
ask_photo_df.head()

,Tags,Questions
0,__label__equipmen __label__recommendation __la...,current community
1,__label__lightroom __label__photo-editing,more stack exchange communities
2,__label__lightroom __label__keywords __label__...,What should I be careful about while buying us...
3,__label__flash __label__hotshoe-flash __label_...,How can I make part of an image transparent in...
4,__label__landscape __label__prime __label__foc...,Confused about Lightroom Smart Collection cond...


In [18]:
ask_photo_tags = ' '.join(ask_photo_df['Tags'].str.lower())

In [19]:
ask_photo_df["Tags"] = [item.replace("|", " __label__") for item in ask_photo_df["Tags"]]
ask_photo_df['Tags'] ='__label__' + ask_photo_df['Tags'].astype(str)
ask_photo_df.head()

,Tags,Questions
0,__label____label__equipmen __label__recommenda...,current community
1,__label____label__lightroom __label__photo-edi...,more stack exchange communities
2,__label____label__lightroom __label__keywords ...,What should I be careful about while buying us...
3,__label____label__flash __label__hotshoe-flash...,How can I make part of an image transparent in...
4,__label____label__landscape __label__prime __l...,Confused about Lightroom Smart Collection cond...


In [20]:
f= open("Photography_txt.txt","w+") #open the file for writing
f.close() #finally closing the file

In [21]:
ask_photo_df.to_csv(r'Photography_txt.txt', header=None, index=None, sep=' ', mode='a')

In [22]:
with open('Photography_txt.txt', 'r') as f, open('photo.txt', 'w') as fo:
    for line in f:
        fo.write(line.replace('"', '').replace("'", ""))

In [23]:
!head photo.txt

__label____label__equipmen __label__recommendation __label__fujifilm __label__used-equipment current community
__label____label__lightroom __label__photo-editing more stack exchange communities
__label____label__lightroom __label__keywords __label__smar __label__collections What should I be careful about while buying used mirrorless camera and lenses?
__label____label__flash __label__hotshoe-flash __label__vintage __label__sync-speed How can I make part of an image transparent in Lightroom?
__label____label__landscape __label__prime __label__focal-distance Confused about Lightroom Smart Collection conditions
__label____label__camera-settings __label__fujifilm Is there a technical reason for modern on-camera flashes flash duration?
__label____label__effect __label__blur Fast Prime Wide lens for Landscape Photography and focal distance [duplicate]
__label____label__nikon __label__live-view How to shortcut the “releasefocus priority” on the FujiFilm X-T4?
__label____label__lighting __labe

In [24]:
!wc photo.txt

   24265  331864 3106498 photo.txt


In [25]:
a= int(round(len(ask_photo_df)*0.75, 0))
print(a)
b= int(round(len(ask_photo_df)*0.25, 0))
b

18199


6066

In [26]:
!head -n 18198 photo.txt > photo.train
!tail -n 6066 photo.txt > photo.valid

In [27]:
pip install fasttext

Note: you may need to restart the kernel to use updated packages.


In [28]:
import fasttext

In [29]:
model = fasttext.train_supervised(input="photo.train")

In [30]:
model.predict("Is there a technical reason for modern on-camera flashes' flash duration?")

(('__label____label__lens',), array([0.0322944]))

In [31]:
model.test("photo.valid")

(6043, 0.13221909647526064, 0.04104381774284687)

In [32]:
model.test("photo.valid", k=5)

(6043, 0.054343868939268576, 0.08434787075563775)

In [33]:
model.predict("Is there a technical reason for modern on-camera flashes' flash duration?", k=5)

(('__label____label__lens',
  '__label____label__canon',
  '__label____label__nikon',
  '__label____label__lightroom',
  '__label____label__film'),
 array([0.0322944 , 0.02655394, 0.01967868, 0.015914  , 0.0133177 ]))

In [34]:
cat photo.txt | sed -e "s/\([.\!?,'/()]\)/ \1 /g" | tr "[:upper:]" "[:lower:]" > photo.preprocessed.txt

In [35]:
!head -n 18198 photo.preprocessed.txt > photo.train

In [36]:
!tail -n 6065 photo.preprocessed.txt > photo.valid

In [37]:
model = fasttext.train_supervised(input="photo.train")

In [38]:
model.test("photo.valid")

(6042, 0.13224097980801058, 0.041056471918195364)

In [39]:
model = fasttext.train_supervised(input="photo.train", epoch=25)

In [40]:
model.test("photo.valid")

(6042, 0.1178417742469381, 0.03658599249781615)

In [41]:
model = fasttext.train_supervised(input="photo.train", lr=0.03)

In [42]:
model.test("photo.valid")

(6042, 0.13224097980801058, 0.041056471918195364)

In [43]:
model = fasttext.train_supervised(input="photo.train", lr=0.03, epoch=50)

In [44]:
model.test("photo.valid")

(6042, 0.13058589870903675, 0.04054262370895637)

In [45]:
model = fasttext.train_supervised(input="photo.train", lr=0.03, epoch=50, wordNgrams=2)

In [46]:
model.test("photo.valid")

(6042, 0.13224097980801058, 0.041056471918195364)

In [47]:
model = fasttext.train_supervised(input="photo.train", lr=0.03, epoch=50, wordNgrams=2, bucket=200000, dim=50, loss='hs')

In [48]:
model.test("photo.valid") #complete

(6042, 0.12528963919232042, 0.038898309439391605)